<a href="https://colab.research.google.com/github/rajitakolla/computational_methods/blob/master/In_class_exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The fifth In-class-exercise (9/30/2020, 20 points in total)

In exercise-03, I asked you to collected 500 textual data based on your own information needs (If you didn't collect the textual data, you should recollect for this exercise). Now we need to think about how to represent the textual data for text classification. In this exercise, you are required to select 10 types of features (10 types of features but absolutely more than 10 features) in the followings feature list, then represent the 500 texts with these features. The output should be in the following format:
![image.png](attachment:image.png)

The feature list:

* (1) tf-idf features
* (2) POS-tag features: number of adjective, adverb, auxiliary, punctuation, complementizer, coordinating conjunction, subordinating conjunction, determiner, interjection, noun, possessor, preposition, pronoun, quantifier, verb, and other. (select some of them if you use pos-tag features)
* (3) Linguistic features:
  * number of right-branching nodes across all constituent types
  * number of right-branching nodes for NPs only
  * number of left-branching nodes across all constituent types
  * number of left-branching nodes for NPs only
  * number of premodifiers across all constituent types
  * number of premodifiers within NPs only
  * number of postmodifiers across all constituent types
  * number of postmodifiers within NPs only
  * branching index across all constituent types, i.e. the number of right-branching nodes minus number of left-branching nodes
  * branching index for NPs only
  * branching weight index: number of tokens covered by right-branching nodes minus number of tokens covered by left-branching nodes across all categories
  * branching weight index for NPs only 
  * modification index, i.e. the number of premodifiers minus the number of postmodifiers across all categories
  * modification index for NPs only
  * modification weight index: length in tokens of all premodifiers minus length in tokens of all postmodifiers across all categories
  * modification weight index for NPs only
  * coordination balance, i.e. the maximal length difference in coordinated constituents
  
  * density (density can be calculated using the ratio of folowing function words to content words) of determiners/quantifiers
  * density of pronouns
  * density of prepositions
  * density of punctuation marks, specifically commas and semicolons
  * density of auxiliary verbs
  * density of conjunctions
  * density of different pronoun types: Wh, 1st, 2nd, and 3rd person pronouns
  
  * maximal and average NP length
  * maximal and average AJP length
  * maximal and average PP length
  * maximal and average AVP length
  * sentence length

* Other features in your mind (ie., pre-defined patterns)

In [151]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [36.0 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/graph

In [153]:
# Write your code here
import urllib.request
import requests 
import pandas as pd
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

In [154]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [220]:
def fetch_data(driver, class_name_val, css_selector_val):
  result = list()
  values = driver.find_elements_by_class_name(class_name_val)
  for value in values:
    result.append(value.find_element(By.CLASS_NAME,css_selector_val).text.replace(" ...\"","").replace("\"...",""))
  return result
  pass

In [221]:
import time
driver.get("https://www.imdb.com/title/tt7286456/reviews?sort=userRating&dir=desc&ratingFilter=0")
#activate the button load_more and run the loop 4 times so to make sure we have 100 words
i=0
while(i<3):
  button = driver.find_element_by_xpath('//*[@id ="load-more-trigger"]').click()
  i = i+1
  time.sleep(2)

reviews = fetch_data(driver, "content", ".text.show-more__control")

ElementNotInteractableException: ignored

In [188]:
required_df = pd.DataFrame(reviews,columns = ['data'])

In [189]:
required_df

,data
0,. Information retrieval addresses the problem...
1,Work in computational linguistics began very ...
2,Abstract-A system that recognizes and authent...
3,Abstract: Testing against natural language re...
4,
5,algorithms that allow understanding and gener...
6,
7,"In recent years, machine learning (ML) has be..."
8,We argue that manual and automatic thesauruse...
9,Introduction Patterns in music have been the ...


In [77]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
list_of_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
frequent_words = list(pd.Series(' '.join(required_df['data']).split()).value_counts()[:15].index)
#top 15 rare words
rare_words = list(pd.Series(' '.join(required_df['data']).split()).value_counts()[-15:].index)

In [79]:
!pip install textblob

In [80]:
from textblob import TextBlob

In [81]:
required_df["punctuations"] = required_df['data'].apply(lambda x:''.join([i for i in x if i not in string.punctuation])).to_frame()
required_df["lower_case"] = required_df['punctuations'].apply(lambda x: x.lower()).to_frame()
required_df["stop_words"]=required_df['lower_case'].apply(lambda x:' '.join([i for i in x.split() if i not in list_of_words])).to_frame()
required_df["frequent_words"] = required_df['stop_words'].apply(lambda x: ' '.join([i for i in x.split() if i not in frequent_words])).to_frame()
required_df["rare_words"] = required_df['frequent_words'].apply(lambda x: ' '.join([i for i in x.split() if i not in rare_words])).to_frame()
required_df["spell_check"]=required_df['rare_words'].apply(lambda x: str(TextBlob(x).correct())).to_frame()


In [82]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [83]:
from nltk import word_tokenize
required_df["tokenization"] = required_df['spell_check'].apply(lambda x: word_tokenize(str(x))).to_frame()

In [84]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
import textblob
ps = PorterStemmer() 

In [85]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [92]:
required_df["stemming"] = required_df['tokenization'].apply(lambda x: ' '.join([ps.stem(i) for i in x])).to_frame()
required_df["final_data"] = required_df['tokenization'].apply(lambda x: ' '.join([textblob.Word(i).lemmatize() for i in x])).to_frame()

required_df

,data,punctuations,lower_case,stop_words,frequent_words,rare_words,spell_check,tokenization,stemming,final_data,word used for TF-IDF,tf,idf,tf-idf
0,5 Ala. 740,5 Ala 740,5 ala 740,5 ala 740,5 ala 740,5 ala 740,5 all 740,"[5, all, 740]",5 all 740,5 all 740,5,0.33,0.48,0.1584
1,Supreme Court of Alabama.,Supreme Court of Alabama,supreme court of alabama,supreme court alabama,supreme court alabama,supreme court alabama,supreme court alabama,"[supreme, court, alabama]",suprem court alabama,supreme court alabama,supreme,0.33,0.48,0.1584
2,ADAMS,ADAMS,adams,adams,adams,adams,adams,[adams],adam,adam,adam,1.00,0.00,0.0000
3,v.,v,v,v,v,v,v,[v],v,v,v,1.00,0.00,0.0000
4,TANNER AND HORTON.,TANNER AND HORTON,tanner and horton,tanner horton,tanner horton,tanner horton,manner norton,"[manner, norton]",manner norton,manner norton,manner,0.50,0.30,0.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,There are no Filings for this citation.,There are no Filings for this citation,there are no filings for this citation,filings citation,filings citation,filings citation,filing situation,"[filing, situation]",file situat,filing situation,filing,0.50,0.30,0.1500
144,Negative Treatment,Negative Treatment,negative treatment,negative treatment,negative treatment,negative treatment,negative treatment,"[negative, treatment]",neg treatment,negative treatment,negative,0.50,0.30,0.1500
145,There are no Negative Treatment results for th...,There are no Negative Treatment results for th...,there are no negative treatment results for th...,negative treatment results citation,negative treatment results citation,negative treatment results citation,negative treatment results situation,"[negative, treatment, results, situation]",neg treatment result situat,negative treatment result situation,negative,0.25,0.60,0.1500
146,History,History,history,history,history,history,history,[history],histori,history,history,1.00,0.00,0.0000


In [131]:
required_df1 = pd.DataFrame(required_df[["final_data"]])
required_df1

,final_data
0,5 all 740
1,supreme court alabama
2,adam
3,v
4,manner norton
...,...
143,filing situation
144,negative treatment
145,negative treatment result situation
146,history


In [132]:
#compute the tf idf and tf-idf values for the starting word in the text

#computeTF
def computeTF(sentence):
  words = sentence.split(" ")
  value = len(set(words))
  return words.count(words[0])/value
  pass


In [133]:
#compute IDF

import math 

def computeIDF(sentence):
  words = sentence.lower().split(" ")
  res = words.count(words[0])
  return math.log(len(words)/res, 10)
  pass

In [134]:

required_df1["word used for TF-IDF"] = required_df1["final_data"].apply(lambda x : x.split(" ")[0])
required_df1["tf"] = required_df1["final_data"].apply(lambda x : round(computeTF(x),2))


In [135]:
required_df1["idf"] = required_df1["final_data"].apply(lambda x : round(computeIDF(x),2))
required_df1["tf-idf"] = required_df1["tf"]*required_df1["idf"]
required_df1

,final_data,word used for TF-IDF,tf,idf,tf-idf
0,5 all 740,5,0.33,0.48,0.1584
1,supreme court alabama,supreme,0.33,0.48,0.1584
2,adam,adam,1.00,0.00,0.0000
3,v,v,1.00,0.00,0.0000
4,manner norton,manner,0.50,0.30,0.1500
...,...,...,...,...,...
143,filing situation,filing,0.50,0.30,0.1500
144,negative treatment,negative,0.50,0.30,0.1500
145,negative treatment result situation,negative,0.25,0.60,0.1500
146,history,history,1.00,0.00,0.0000


In [136]:
import nltk

In [137]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [138]:

lemmatized = required_df1["final_data"].values.tolist()
pos=[]
count=0
for i in lemmatized:
  text = word_tokenize(i)
  count=count+1
  pos.append(nltk.pos_tag(text))    #Parts of Speech for each sentence
pos

[[('5', 'CD'), ('all', 'DT'), ('740', 'CD')],
 [('supreme', 'JJ'), ('court', 'NN'), ('alabama', 'NN')],
 [('adam', 'NN')],
 [('v', 'NN')],
 [('manner', 'NN'), ('norton', 'NN')],
 [('june', 'JJ'), ('term', 'NN'), ('1843', 'CD')],
 [('sycosis', 'NN')],
 [('writ', 'NN'),
  ('error', 'NN'),
  ('circuit', 'NN'),
  ('court', 'NN'),
  ('sumter', 'NN')],
 [],
 [('west', 'NN'), ('headnotes', 'VBZ'), ('2', 'CD')],
 [('1', 'CD')],
 [('chattel', 'NN'), ('mortgage', 'NN')],
 [('crop', 'NN')],
 [('growing', 'VBG'),
  ('crop', 'NN'),
  ('existence', 'NN'),
  ('subjectmatter', 'NN'),
  ('mortgage', 'NN'),
  ('contract', 'NN'),
  ('pass', 'VBP'),
  ('interest', 'NN'),
  ('vest', 'NN'),
  ('possession', 'NN'),
  ('either', 'CC'),
  ('immediately', 'RB'),
  ('future', 'JJ'),
  ('time', 'NN')],
 [('4', 'CD'), ('case', 'NN'), ('cite', 'JJ'), ('headnote', 'NN')],
 [('2', 'CD')],
 [('creditor', 'NN'), ('’', 'CD'), ('remedy', 'NN')],
 [('line', 'NN'), ('priority', 'NN')],
 [('st1821', 'NN'),
  ('prohibiting',

In [139]:
from collections import Counter

def get_pos(value,pos):
  pos_counts=[]
  for j in pos:
    counts = Counter(tag for word,tag in j)   
    pos_counts.append(counts.get(value))
  return pos_counts


In [149]:
required_df1["Number of nouns"] = pd.DataFrame(get_pos("NN",pos))
required_df1["Number of adverb"] = pd.DataFrame(get_pos("RB",pos))
required_df1["Number of cardinal digits"] = pd.DataFrame(get_pos("CD", pos))
required_df1["Number of Verbs"] = pd.DataFrame(get_pos("VB",pos))
required_df1["Number of Determiners"] = pd.DataFrame(get_pos("DT",pos)) 
required_df1["Number of adjectives"] = pd.DataFrame(get_pos("JJ",pos)) 
required_df1["Number of interjenctions"] = pd.DataFrame(get_pos("UH",pos))

In [150]:
required_df1

,final_data,word used for TF-IDF,tf,idf,tf-idf,Number of nouns,Number of adverb,Number of cardinal digits,Number of Verbs,Number of Determiners,Number of adjectives,Number of interjenctions
0,5 all 740,5,0.33,0.48,0.1584,NaN,NaN,2.0,NaN,1.0,NaN,None
1,supreme court alabama,supreme,0.33,0.48,0.1584,2.0,NaN,NaN,NaN,NaN,1.0,None
2,adam,adam,1.00,0.00,0.0000,1.0,NaN,NaN,NaN,NaN,NaN,None
3,v,v,1.00,0.00,0.0000,1.0,NaN,NaN,NaN,NaN,NaN,None
4,manner norton,manner,0.50,0.30,0.1500,2.0,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...
143,filing situation,filing,0.50,0.30,0.1500,1.0,NaN,NaN,NaN,NaN,NaN,None
144,negative treatment,negative,0.50,0.30,0.1500,1.0,NaN,NaN,NaN,NaN,1.0,None
145,negative treatment result situation,negative,0.25,0.60,0.1500,3.0,NaN,NaN,NaN,NaN,1.0,None
146,history,history,1.00,0.00,0.0000,1.0,NaN,NaN,NaN,NaN,NaN,None
